In [ ]:
import bs4
import os
from dotenv import load_dotenv, dotenv_values
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, StateGraph

env_path = '.env'
env_vars = dotenv_values(env_path)
for key in env_vars:
    if key in os.environ:
        del os.environ[key]

# Explicitly reload the .env file
load_dotenv(env_path)

os.environ["LANGCHAIN_PROJECT"] = "conversational-test"
print(os.environ["LANGCHAIN_PROJECT"])


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


with SqliteSaver.from_conn_string(":memory:") as memory:
   ### Build retriever tool ###
    tool = create_retriever_tool(
        retriever,
        "blog_post_retriever",
        "Searches and returns excerpts from the Autonomous Agents blog post.",
    )
    tools = [tool]

    agent_executor = create_react_agent(llm, tools, checkpointer=memory)
    
    query = "What is Task Decomposition?"

    config = {"configurable": {"thread_id": "abc123"}}
    
    for s in agent_executor.stream(
        {"messages": [HumanMessage(content="Hi! I'm bob")]}, config=config
    ):
        print(s)
        print("----")

    query = "What is Task Decomposition?"

    for s in agent_executor.stream(
        {"messages": [HumanMessage(content=query)]}, config=config
    ):
        print(s)
        print("----")
        